In [184]:
from __future__ import absolute_import, division, print_function
from matplotlib.font_manager import _rebuild; _rebuild()
import tensorflow as tf
import re
#Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.io as spio
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import neural_network
tf.logging.set_verbosity(tf.logging.INFO)

In [173]:
"""Load the dataset and set randomness."""

# Initialize random number generator for reproducibility.
seed = 7
np.random.seed(seed)

# Load in dataset.
data = spio.loadmat("features_10s_2019-01-30.mat");
features = data['features'];
labels = data['labels_features'];
animal_id_features = data['animal_id_features'].transpose();

feat_names = data['feat_names']
col_names = pd.DataFrame(feat_names) 
# Label each feature column with its description.
def find_between(s):
    start = '\'';
    end = '\'';
    return((s.split(start))[1].split(end)[0])
cols = [];
c_names = col_names.values.ravel();

for x in range(len(c_names)):
    name = str (c_names[x]);
    cols.append(find_between(name))
    
# Create a DataFrame of features with columns named & rows labeled.    
feat_data = pd.DataFrame(data=features,columns=cols)
feat_data.insert(0,'AnimalId',animal_id_features)
feat_data.insert(0,'Labels',labels.transpose())

# Select the features corresponding to one animal.
def get_single_animal_feats(df,index) : 
    return df.loc[df['AnimalId'] == index]

# TODO: Randomize!
# For now, we select the features of the first animal.
fKH41_features = get_single_animal_feats(feat_data, 1)

# Get only labels corresponding to first animal's features.
y = fKH41_features['Labels']
X = fKH41_features.drop(columns={'Labels','AnimalId'})

In [176]:
"""Split data into training and testing for cross-validation."""
X_train, X_test, y_train, y_test = train_test_split(X, y);

In [189]:
"""Standardize the data since the MLP is sensitive to feature scaling."""
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# Fit only to the training data.
scaler.fit(X_train)
# Apply the transformations to the data.
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

"""Training the model!"""
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes(10,10,10),max_iter=500)

NameError: name 'hidden_layer_sizes' is not defined

In [186]:
"""Define the neural network model with 141 inputs, x hidden nodes, 
and 3 outputs (pre-ictal, inter-ictal, normal)."""
def nn_model():
    # Create model.
    model = Sequential()
    model.add(Dense(8,input_dim=141,activation='relu'))
    model.add(Dense(units=3,activation='softmax'))
    # Compile model.
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    return model

In [ ]:
data.tags.values

In [50]:
"""Build the model"""
from keras.wrappers.scikit_learn import KerasClassifier, OneVsAllClais
# keras_model = nn_model()
# keras_model.fit(data_train, labels_train, epochs=50, batch_size=100, verbose=1)

"""Build a OneVsRest classifier to obtain ROC"""
classifier = OneVsRestClassifier(LinearSVC(random_state=seed))
y_score = classifier.fit(data_train, labels_train).decision_function(data_test)

ImportError: cannot import name 'OneVsAllClassifier'

In [47]:
"""Evaluate performance by false positive & true positive rates."""
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(test_data[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

ValueError: continuous-multioutput format is not supported

In [ ]:
"""Create KerasClassifier and pass args to the fit function used internally to train the NN."""
# Set batch size, steps per epoch.
training_set_size = data_train.shape[0];
batch_size = 1000
steps_per_epoch = (training_set_size // batch_size)
print("NN Args: Batch Size: {}, Steps/Epoch: {}".format(batch_size, steps_per_epoch));

# Compile the model and record history.
estimator = KerasClassifier(build_fn=nn_model,epochs=50,batch_size=batch_size,verbose=0)
history = estimator.fit(data_train,labels_train,verbose=0)

# # Summarize history for accuracy.
# plt.plot(history.history['acc'])
# # plt.plot(history.history['val_acc'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()

# # Summarize history for loss.
# plt.plot(history.history['loss'])
# # plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'test'], loc='upper left')
# plt.show()


In [ ]:
# Evaluate results using cross-validation.
kfold = KFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(estimator, data_train, labels_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))